In [1]:
import numpy as np
import pandas as pd

from pandas import Series, DataFrame
import urllib.request
from bs4 import BeautifulSoup

import time
import os

webpage = "https://www.foodnetwork.com/recipes/ina-garten/16-bean-pasta-e-fagioli-1-3753755"

#This returns HTTPResponse object need to read and  
#decode to the see the contents.
html_bytes = urllib.request.urlopen(webpage)

#To decode the bytes to a string this is the HTML of the webpage
#html_string = html_bytes.read().decode("utf-8")

#This is the soup object that will parse th HTML page
soup = BeautifulSoup(html_bytes,"html5lib")

In [2]:
#Recipe Title
print(soup.title)
title=soup.title.string[:soup.title.string.find('|')].rstrip()
title


<title>"16 Bean" Pasta e Fagioli Recipe | Ina Garten | Food Network</title>


'"16 Bean" Pasta e Fagioli Recipe'

In [11]:
#extract ingredient list

start=soup.get_text().find('"recipeIngredient": [')
end=start+soup.get_text()[soup.get_text().find('"recipeIngredient": ['):].find(']')
s=soup.get_text()[start+27:end]
print(s)
print(s.split(','))
ingredient=[i for i in s.split('",\n    "')]
ingredient[-1]=ingredient[-1][:-4]
ingredient

1 (1-pound) bag 16 Bean Soup Mix",
    "2 tablespoons good olive oil, plus extra for serving",
    "6 ounces pancetta, 1/4-inch-diced",
    "1 large onion, chopped",
    "1 tablespoon minced garlic (3 cloves)",
    "1/2 teaspoon crushed red pepper flakes",
    "1 (28-ounce) can crushed tomatoes",
    "1 cup dry red wine",
    "4 to 6 cups good chicken stock, preferably homemade",
    "Kosher salt and freshly ground black pepper",
    "1 cup miniature pasta, such as ditalini or tubettini",
    "1/2 cup freshly grated Italian Parmesan cheese, plus extra for serving",
    "1 tablespoon good red wine vinegar",
    "Julienned fresh basil leaves, for serving"
  
['1 (1-pound) bag 16 Bean Soup Mix"', '\n    "2 tablespoons good olive oil', ' plus extra for serving"', '\n    "6 ounces pancetta', ' 1/4-inch-diced"', '\n    "1 large onion', ' chopped"', '\n    "1 tablespoon minced garlic (3 cloves)"', '\n    "1/2 teaspoon crushed red pepper flakes"', '\n    "1 (28-ounce) can crushed tomatoes"', '

['1 (1-pound) bag 16 Bean Soup Mix',
 '2 tablespoons good olive oil, plus extra for serving',
 '6 ounces pancetta, 1/4-inch-diced',
 '1 large onion, chopped',
 '1 tablespoon minced garlic (3 cloves)',
 '1/2 teaspoon crushed red pepper flakes',
 '1 (28-ounce) can crushed tomatoes',
 '1 cup dry red wine',
 '4 to 6 cups good chicken stock, preferably homemade',
 'Kosher salt and freshly ground black pepper',
 '1 cup miniature pasta, such as ditalini or tubettini',
 '1/2 cup freshly grated Italian Parmesan cheese, plus extra for serving',
 '1 tablespoon good red wine vinegar',
 'Julienned fresh basil leaves, for serving']

In [12]:
#extract directions 

start=soup.get_text().find('"recipeInstructions": [')
end=start+soup.get_text()[soup.get_text().find('"recipeInstructions": ['):].find(']')
s=soup.get_text()[start:end]
directions=[i.rstrip() for i in s[len('"recipeInstructions": '):].split('\n')][1:-1]
directions

['    "The day before you plan to make the soup, place the bean mix in a large bowl, add cold water to cover by 2 inches, and refrigerate overnight. The next day, drain the beans, rinse under cold running water, and drain again. Place the beans in a large pot with 8 cups of cold water. Bring to a boil, lower the heat, and simmer for 1 hour. Stir occasionally and skim off any foam that rises to the top. The beans should be very tender and the skin will peel away when you blow on a bean.",',
 '    "Meanwhile, heat the oil in a medium (10-inch) stockpot or Dutch oven over medium heat. Add the pancetta and onion and saute over medium to medium-high heat for 12 to 18 minutes, until browned. Add the garlic and red pepper flakes and saute for one minute. Add the tomatoes, wine, 4 cups of the chicken stock, 1 tablespoon salt, and 1 teaspoon black pepper and turn off the heat.",',
 '    "Drain the beans and add two-thirds of them to the soup. Pass the remaining beans through a food mill, discar

In [13]:
# extract rating information

start=soup.get_text().find('"aggregateRating": {')
print(start)
end=start+soup.get_text()[soup.get_text().find('"aggregateRating": {'):].find('}')
s=soup.get_text()[start:end]
print(s)
reviewCount=int(s.split(',')[2].rstrip()[20:])
print('review count: ',reviewCount)
rating=s.split(',')[1].rstrip()[20:]
print('rating: ',rating)

10685
"aggregateRating": {
    "@type": "AggregateRating",
    "ratingValue": 5,
    "reviewCount": 3
    
review count:  3
rating:  5


In [7]:
start=soup.get_text().find('"recipeCategory": [')
end=start+soup.get_text()[soup.get_text().find('"recipeCategory": ['):].find(']')
s=soup.get_text()[start:end]
print(s)
category=[i for i in s[25:].split('",\n    "')]
category[-1]=category[-1][:-4]
category

"recipeCategory": [
    "Italian",
    "Soup",
    "Beans and Legumes"
  


['Italian', 'Soup', 'Beans and Legumes']

In [14]:
# extract time

start=soup.get_text().find('"cookTime": ')
end=start+soup.get_text()[soup.get_text().find('"cookTime": '):].find('"totalTime"')
s=soup.get_text()[start:soup.get_text().find('S',end)+1]
print(s)
cookTime=s.split(',')[0].rstrip()[13:-1]
if(s.find('prepTime')!=-1):
    prepTime=s.split(',')[1].rstrip()[16:-1]
    prepTime= int(prepTime[prepTime.find('T')+1:prepTime.find('H',prepTime.find('T'))])*60+int(prepTime[prepTime.find('H')+1:prepTime.find('M',prepTime.find('H'))])
    totalTime=s.split(',')[2].rstrip()[16:-1]
else:
    totalTime=s.split(',')[1].rstrip()[16:-1]
    prepTime=0

cookTime= int(cookTime[cookTime.find('T')+1:cookTime.find('H',cookTime.find('T'))])*60+int(cookTime[cookTime.find('H')+1:cookTime.find('M',cookTime.find('H'))])
totalTime=int(totalTime[totalTime.find('T')+1:totalTime.find('H',totalTime.find('T'))])*60+int(totalTime[totalTime.find('H')+1:totalTime.find('M',totalTime.find('H'))])

print('cook time ', cookTime, 'minutes')
print('prep time ',prepTime, 'minutes')
print('total time ',totalTime, 'minutes')

"cookTime": "P0Y0M0DT0H4M0.000S",
  "totalTime": "P0Y0M0DT10H10M0.000S
cook time  4 minutes
prep time  0 minutes
total time  610 minutes


In [16]:
# create Series

recipies=DataFrame()
columns=[ingredient,category,rating,reviewCount,directions,cookTime,prepTime]
labels=['ingredients','categories','rating','review count','directions','cook time','prep time']
s=Series(columns,index=labels)
recipies[title]=s
r=recipies.transpose()
r

,ingredients,categories,rating,review count,directions,cook time,prep time
"""16 Bean"" Pasta e Fagioli Recipe","[1 (1-pound) bag 16 Bean Soup Mix, 2 tablespoo...","[Italian, Soup, Beans and Legumes]",5,3,"[ ""The day before you plan to make the soup...",4,0


In [17]:
r.directions[0]

['    "The day before you plan to make the soup, place the bean mix in a large bowl, add cold water to cover by 2 inches, and refrigerate overnight. The next day, drain the beans, rinse under cold running water, and drain again. Place the beans in a large pot with 8 cups of cold water. Bring to a boil, lower the heat, and simmer for 1 hour. Stir occasionally and skim off any foam that rises to the top. The beans should be very tender and the skin will peel away when you blow on a bean.",',
 '    "Meanwhile, heat the oil in a medium (10-inch) stockpot or Dutch oven over medium heat. Add the pancetta and onion and saute over medium to medium-high heat for 12 to 18 minutes, until browned. Add the garlic and red pepper flakes and saute for one minute. Add the tomatoes, wine, 4 cups of the chicken stock, 1 tablespoon salt, and 1 teaspoon black pepper and turn off the heat.",',
 '    "Drain the beans and add two-thirds of them to the soup. Pass the remaining beans through a food mill, discar